In [178]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [179]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [180]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [181]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../saved_models/{model_name}'))

# Train on Real Test on Real

In [182]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [183]:
activities = [1, 3, 4]
users = [1, 3, 5]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [184]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [185]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.452085971832275, Final Batch Loss: 2.2119810581207275
Epoch 2, Loss: 4.451161623001099, Final Batch Loss: 2.2264087200164795
Epoch 3, Loss: 4.443710803985596, Final Batch Loss: 2.2184813022613525
Epoch 4, Loss: 4.442732334136963, Final Batch Loss: 2.2178618907928467
Epoch 5, Loss: 4.442281723022461, Final Batch Loss: 2.2302377223968506
Epoch 6, Loss: 4.439357280731201, Final Batch Loss: 2.230043411254883
Epoch 7, Loss: 4.420681476593018, Final Batch Loss: 2.195150375366211
Epoch 8, Loss: 4.412769317626953, Final Batch Loss: 2.188819408416748
Epoch 9, Loss: 4.4177327156066895, Final Batch Loss: 2.2168095111846924
Epoch 10, Loss: 4.420544624328613, Final Batch Loss: 2.2267842292785645
Epoch 11, Loss: 4.404106616973877, Final Batch Loss: 2.203859806060791
Epoch 12, Loss: 4.3958775997161865, Final Batch Loss: 2.2021379470825195
Epoch 13, Loss: 4.384618520736694, Final Batch Loss: 2.188055992126465
Epoch 14, Loss: 4.378288745880127, Final Batch Loss: 2.19391131401062
Epoch 

Epoch 125, Loss: 2.0532474517822266, Final Batch Loss: 0.9920840263366699
Epoch 126, Loss: 2.020070195198059, Final Batch Loss: 1.0039656162261963
Epoch 127, Loss: 1.934305489063263, Final Batch Loss: 0.9073588252067566
Epoch 128, Loss: 2.0967047214508057, Final Batch Loss: 1.1043118238449097
Epoch 129, Loss: 2.0154263973236084, Final Batch Loss: 0.9715622663497925
Epoch 130, Loss: 1.8752815127372742, Final Batch Loss: 0.8571147322654724
Epoch 131, Loss: 1.8935051560401917, Final Batch Loss: 0.8968578577041626
Epoch 132, Loss: 1.9150832891464233, Final Batch Loss: 0.969791829586029
Epoch 133, Loss: 1.9160397052764893, Final Batch Loss: 0.983177661895752
Epoch 134, Loss: 1.9315056800842285, Final Batch Loss: 0.9388752579689026
Epoch 135, Loss: 1.907244861125946, Final Batch Loss: 1.0310388803482056
Epoch 136, Loss: 1.9086102843284607, Final Batch Loss: 0.9589402675628662
Epoch 137, Loss: 1.8526723384857178, Final Batch Loss: 0.9814061522483826
Epoch 138, Loss: 1.9305467009544373, Final 

Epoch 244, Loss: 1.12730073928833, Final Batch Loss: 0.5555812120437622
Epoch 245, Loss: 1.0885522961616516, Final Batch Loss: 0.509425699710846
Epoch 246, Loss: 1.0819045305252075, Final Batch Loss: 0.5658247470855713
Epoch 247, Loss: 1.1220088601112366, Final Batch Loss: 0.5802668929100037
Epoch 248, Loss: 1.0327616333961487, Final Batch Loss: 0.5115811228752136
Epoch 249, Loss: 1.1485384106636047, Final Batch Loss: 0.5937907695770264
Epoch 250, Loss: 1.0358011722564697, Final Batch Loss: 0.5186923146247864
Epoch 251, Loss: 1.2267341017723083, Final Batch Loss: 0.543980598449707
Epoch 252, Loss: 1.14228755235672, Final Batch Loss: 0.6066933870315552
Epoch 253, Loss: 1.0621596574783325, Final Batch Loss: 0.5413356423377991
Epoch 254, Loss: 1.1309849619865417, Final Batch Loss: 0.5884868502616882
Epoch 255, Loss: 1.0735015869140625, Final Batch Loss: 0.4852635860443115
Epoch 256, Loss: 1.0189102292060852, Final Batch Loss: 0.5033032894134521
Epoch 257, Loss: 1.1265581846237183, Final B

Epoch 369, Loss: 0.911255270242691, Final Batch Loss: 0.46673843264579773
Epoch 370, Loss: 0.9503285884857178, Final Batch Loss: 0.5461252927780151
Epoch 371, Loss: 0.9270475804805756, Final Batch Loss: 0.4909852147102356
Epoch 372, Loss: 0.9125403761863708, Final Batch Loss: 0.5033171772956848
Epoch 373, Loss: 0.8852497637271881, Final Batch Loss: 0.3642512261867523
Epoch 374, Loss: 0.8929900825023651, Final Batch Loss: 0.4815034866333008
Epoch 375, Loss: 0.9288673102855682, Final Batch Loss: 0.48586538434028625
Epoch 376, Loss: 0.9032082855701447, Final Batch Loss: 0.4820229113101959
Epoch 377, Loss: 0.8832864761352539, Final Batch Loss: 0.39674755930900574
Epoch 378, Loss: 0.8555938899517059, Final Batch Loss: 0.45116081833839417
Epoch 379, Loss: 0.8440591990947723, Final Batch Loss: 0.42449888586997986
Epoch 380, Loss: 0.8295158743858337, Final Batch Loss: 0.33106571435928345
Epoch 381, Loss: 0.9004021286964417, Final Batch Loss: 0.3983585238456726
Epoch 382, Loss: 0.91558825969696

Epoch 486, Loss: 0.8636487424373627, Final Batch Loss: 0.4521697759628296
Epoch 487, Loss: 0.8343479335308075, Final Batch Loss: 0.45585906505584717
Epoch 488, Loss: 0.8109950721263885, Final Batch Loss: 0.39529648423194885
Epoch 489, Loss: 0.8389362692832947, Final Batch Loss: 0.3829059600830078
Epoch 490, Loss: 0.7704326510429382, Final Batch Loss: 0.38216185569763184
Epoch 491, Loss: 0.727622002363205, Final Batch Loss: 0.37106141448020935
Epoch 492, Loss: 0.7441194951534271, Final Batch Loss: 0.35205936431884766
Epoch 493, Loss: 0.8418553173542023, Final Batch Loss: 0.49109789729118347
Epoch 494, Loss: 0.7780665457248688, Final Batch Loss: 0.3630353808403015
Epoch 495, Loss: 0.7771190106868744, Final Batch Loss: 0.3900231719017029
Epoch 496, Loss: 0.776375949382782, Final Batch Loss: 0.44187963008880615
Epoch 497, Loss: 0.8295967876911163, Final Batch Loss: 0.42291295528411865
Epoch 498, Loss: 0.7851683497428894, Final Batch Loss: 0.38400837779045105
Epoch 499, Loss: 0.841899067163

...

Epoch 4450, Loss: 0.18286597728729248, Final Batch Loss: 0.07551492005586624
Epoch 4451, Loss: 0.27844327688217163, Final Batch Loss: 0.13677754998207092
Epoch 4452, Loss: 0.2211771234869957, Final Batch Loss: 0.11370570957660675
Epoch 4453, Loss: 0.17841382324695587, Final Batch Loss: 0.06839116662740707
Epoch 4454, Loss: 0.2089892029762268, Final Batch Loss: 0.12137484550476074
Epoch 4455, Loss: 0.14507203549146652, Final Batch Loss: 0.0627676472067833
Epoch 4456, Loss: 0.21507905423641205, Final Batch Loss: 0.12862151861190796
Epoch 4457, Loss: 0.1761307492852211, Final Batch Loss: 0.07703227549791336
Epoch 4458, Loss: 0.21796602755784988, Final Batch Loss: 0.07797285169363022
Epoch 4459, Loss: 0.14531387388706207, Final Batch Loss: 0.06835572421550751
Epoch 4460, Loss: 0.17441987991333008, Final Batch Loss: 0.07933668047189713
Epoch 4461, Loss: 0.20844805240631104, Final Batch Loss: 0.09088282287120819
Epoch 4462, Loss: 0.17581962049007416, Final Batch Loss: 0.07527457177639008
Epo

Epoch 4561, Loss: 0.18154702335596085, Final Batch Loss: 0.06359310448169708
Epoch 4562, Loss: 0.20515019446611404, Final Batch Loss: 0.08159332722425461
Epoch 4563, Loss: 0.2317010909318924, Final Batch Loss: 0.12609738111495972
Epoch 4564, Loss: 0.17368608713150024, Final Batch Loss: 0.10055797547101974
Epoch 4565, Loss: 0.16246102005243301, Final Batch Loss: 0.09463146328926086
Epoch 4566, Loss: 0.2129187509417534, Final Batch Loss: 0.12176142632961273
Epoch 4567, Loss: 0.2571898326277733, Final Batch Loss: 0.09424792975187302
Epoch 4568, Loss: 0.1969277337193489, Final Batch Loss: 0.09879229962825775
Epoch 4569, Loss: 0.2000475972890854, Final Batch Loss: 0.09165756404399872
Epoch 4570, Loss: 0.20821496099233627, Final Batch Loss: 0.12447801232337952
Epoch 4571, Loss: 0.21478526294231415, Final Batch Loss: 0.10727684199810028
Epoch 4572, Loss: 0.22511909902095795, Final Batch Loss: 0.12166096270084381
Epoch 4573, Loss: 0.22881296277046204, Final Batch Loss: 0.10945528745651245
Epoc

Epoch 4672, Loss: 0.21754104644060135, Final Batch Loss: 0.1304243505001068
Epoch 4673, Loss: 0.23417479544878006, Final Batch Loss: 0.15166904032230377
Epoch 4674, Loss: 0.23818986117839813, Final Batch Loss: 0.09916380047798157
Epoch 4675, Loss: 0.16400843113660812, Final Batch Loss: 0.091618113219738
Epoch 4676, Loss: 0.20455146580934525, Final Batch Loss: 0.09217371791601181
Epoch 4677, Loss: 0.21625296771526337, Final Batch Loss: 0.14682511985301971
Epoch 4678, Loss: 0.2504638582468033, Final Batch Loss: 0.1373957097530365
Epoch 4679, Loss: 0.22915935516357422, Final Batch Loss: 0.09336104989051819
Epoch 4680, Loss: 0.15808677673339844, Final Batch Loss: 0.07114493101835251
Epoch 4681, Loss: 0.18245181441307068, Final Batch Loss: 0.10363525152206421
Epoch 4682, Loss: 0.24583376944065094, Final Batch Loss: 0.10021248459815979
Epoch 4683, Loss: 0.28540556132793427, Final Batch Loss: 0.14305591583251953
Epoch 4684, Loss: 0.17736519128084183, Final Batch Loss: 0.10539324581623077
Epoc

Epoch 4784, Loss: 0.19919109344482422, Final Batch Loss: 0.07493841648101807
Epoch 4785, Loss: 0.19797784835100174, Final Batch Loss: 0.10192470252513885
Epoch 4786, Loss: 0.20949847996234894, Final Batch Loss: 0.1044020876288414
Epoch 4787, Loss: 0.1867453157901764, Final Batch Loss: 0.09250257909297943
Epoch 4788, Loss: 0.15220796316862106, Final Batch Loss: 0.06684347987174988
Epoch 4789, Loss: 0.21382589638233185, Final Batch Loss: 0.1008482426404953
Epoch 4790, Loss: 0.21625323593616486, Final Batch Loss: 0.0838804543018341
Epoch 4791, Loss: 0.23402804136276245, Final Batch Loss: 0.11028219759464264
Epoch 4792, Loss: 0.14107517898082733, Final Batch Loss: 0.0625104084610939
Epoch 4793, Loss: 0.24429992586374283, Final Batch Loss: 0.09181607514619827
Epoch 4794, Loss: 0.22761959582567215, Final Batch Loss: 0.08858095854520798
Epoch 4795, Loss: 0.21310542523860931, Final Batch Loss: 0.10143326967954636
Epoch 4796, Loss: 0.17421536147594452, Final Batch Loss: 0.09902460873126984
Epoc

Epoch 4897, Loss: 0.16347705572843552, Final Batch Loss: 0.07901841402053833
Epoch 4898, Loss: 0.17880918085575104, Final Batch Loss: 0.08442215621471405
Epoch 4899, Loss: 0.2553791478276253, Final Batch Loss: 0.1805746704339981
Epoch 4900, Loss: 0.16907446458935738, Final Batch Loss: 0.11507628858089447
Epoch 4901, Loss: 0.17990579456090927, Final Batch Loss: 0.0640285462141037
Epoch 4902, Loss: 0.2142663449048996, Final Batch Loss: 0.13544540107250214
Epoch 4903, Loss: 0.21623771637678146, Final Batch Loss: 0.10612701624631882
Epoch 4904, Loss: 0.25547612458467484, Final Batch Loss: 0.15996429324150085
Epoch 4905, Loss: 0.15684397518634796, Final Batch Loss: 0.07867634296417236
Epoch 4906, Loss: 0.17463793605566025, Final Batch Loss: 0.11192311346530914
Epoch 4907, Loss: 0.14774642139673233, Final Batch Loss: 0.05794867128133774
Epoch 4908, Loss: 0.13298435509204865, Final Batch Loss: 0.06449439376592636
Epoch 4909, Loss: 0.22617466747760773, Final Batch Loss: 0.0908125787973404
Epoc

In [186]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[18  0  0  0  0  0  0  0  0]
 [ 0 11  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0]
 [ 0  0  0  1 14  0  0  0  0]
 [ 0  0  1  0  0  6  0  0  0]
 [ 0  0  0  0  0  0 11  0  0]
 [ 0  0  0  0  0  0  0 10  0]
 [ 0  0  0  0  0  1  0  0  7]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        18
           1    1.00000   1.00000   1.00000        11
           2    0.90909   1.00000   0.95238        10
           3    0.90909   1.00000   0.95238        10
           4    1.00000   0.93333   0.96552        15
           5    0.85714   0.85714   0.85714         7
           6    1.00000   1.00000   1.00000        11
           7    1.00000   1.00000   1.00000        10
           8    1.00000   0.87500   0.93333         8

    accuracy                        0.97000       100
   macro avg    0.96392   0.96283   0.96231       100
weighted avg    0.97182   0.97000   0.96997       100



# Train on Fake Test on Real

In [ ]:
# 20,000 epochs DEFAULT PARAMETERS FOR THRESHOLDS

In [187]:
gen = Generator(z_dim = 106)
load_model(gen, "cGAN_UCI_8_19_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=106, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=46, bias=True)
    (4): Tanh()
  )
)

In [201]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 3)

fake_labels = []

for k in range(size):
    if act_vectors[0][k] == 0 and usr_vectors[0][k] == 0:
        fake_labels.append(0)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 0:
        fake_labels.append(1)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 0:
        fake_labels.append(2)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 1:
        fake_labels.append(3)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 1:
        fake_labels.append(4)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 1:
        fake_labels.append(5)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 2:
        fake_labels.append(6)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 2:
        fake_labels.append(7)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 2:
        fake_labels.append(8)

fake_labels = np.asarray(fake_labels)
to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()
#fake_features = gen(to_gen).detach().numpy()

In [202]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[13  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0]
 [ 0  0  0 15  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  1]
 [ 0  0  0  0  0  0 11  0  0]
 [ 0  0  0  0  0  0  0 16  0]
 [ 0  0  0  0  0  0  2  0  0]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        13
           1    1.00000   1.00000   1.00000         5
           2    1.00000   1.00000   1.00000        14
           3    1.00000   1.00000   1.00000        15
           4    1.00000   1.00000   1.00000        10
           5    1.00000   0.92857   0.96296        14
           6    0.84615   1.00000   0.91667        11
           7    1.00000   1.00000   1.00000        16
           8    0.00000   0.00000   0.00000         2

    accuracy                        0.97000       100
   macro avg    0.87179   0.88095   0.87551       100
weighted avg    0.96308   0.97000   0.96565       100

